In [11]:
from google.colab import files

# Upload the ZIP file
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [15]:
import os

# List the current directory to ensure kaggle.json is there
print(os.listdir())


['.config', '.ipynb_checkpoints', 'kaggle.json', 'sample_data']


In [16]:
import os

# Make a directory for the Kaggle configuration
os.makedirs('/root/.kaggle', exist_ok=True)

# Move kaggle.json to the directory
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json


In [17]:
# Replace with your dataset path from Kaggle
!kaggle datasets download -d msambare/fer2013


Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
 96% 58.0M/60.3M [00:03<00:00, 25.0MB/s]
100% 60.3M/60.3M [00:03<00:00, 18.5MB/s]


In [18]:
import zipfile

# Extract the downloaded dataset
with zipfile.ZipFile('fer2013.zip', 'r') as zip_ref:
    zip_ref.extractall('./fer2013')

print("Dataset extracted!")


Dataset extracted!


In [22]:
import os

# List the files in the extracted directory
print(os.listdir('./fer2013'))
print(os.listdir('./fer2013/train'))  # Check the 'train' folder if it exists
print(os.listdir('./fer2013/test'))   # Check the 'test' folder if it exists


['test', 'train']
['fear', 'surprise', 'happy', 'angry', 'disgust', 'neutral', 'sad']
['fear', 'surprise', 'happy', 'angry', 'disgust', 'neutral', 'sad']


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to train and test directories
train_dir = './fer2013/train'
test_dir = './fer2013/test'

# Create ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # Resize all images to 48x48
    color_mode='grayscale',  # Use grayscale images
    batch_size=32,
    class_mode='categorical'  # For multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),  # Resize all images to 48x48
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [25]:
print("Class indices:", train_generator.class_indices)


Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 95s 104ms/step - accuracy: 0.2761 - loss: 1.7884 - val_accuracy: 0.4161 - val_loss: 1.5373
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 92s 102ms/step - accuracy: 0.3985 - loss: 1.5467 - val_accuracy: 0.4549 - val_loss: 1.4341
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 146s 107ms/step - accuracy: 0.4445 - loss: 1.4511 - val_accuracy: 0.4746 - val_loss: 1.3592
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 95s 105ms/step - accuracy: 0.4625 - loss: 1.4056 - val_accuracy: 0.4896 - val_loss: 1.3300
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 140s 103ms/step - accuracy: 0.4782 - loss: 1.3624 - val_accuracy: 0.4936 - val_loss: 1.3112
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 94s 105ms/step - accuracy: 0.5002 - loss: 1.3063 - val_accuracy: 0.5086 - val_loss: 1.2807
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 142s 104ms/step - accuracy: 0.5036 - loss: 1.2846 - val_accuracy: 0.5065 - val_loss: 1.2695
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 95s 106ms/step - accuracy: 0.5263 - loss

In [29]:
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=test_generator
)


Epoch 1/3
898/898 ━━━━━━━━━━━━━━━━━━━━ 88s 98ms/step - accuracy: 0.5682 - loss: 1.1269 - val_accuracy: 0.5298 - val_loss: 1.2276
Epoch 2/3
898/898 ━━━━━━━━━━━━━━━━━━━━ 91s 101ms/step - accuracy: 0.5762 - loss: 1.0945 - val_accuracy: 0.5320 - val_loss: 1.2255
Epoch 3/3
898/898 ━━━━━━━━━━━━━━━━━━━━ 89s 99ms/step - accuracy: 0.5843 - loss: 1.0692 - val_accuracy: 0.5313 - val_loss: 1.2378


In [30]:
model.save('emotion_recognition_model.h5')


In [32]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


225/225 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.5324 - loss: 1.2333
Test Loss: 1.2378313541412354
Test Accuracy: 0.531345784664154


In [33]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('emotion_recognition_model.h5')


In [37]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to the image you want to predict
img_path = '/content/OIP (1).jpg'

# Load and preprocess the image
img = image.load_img(img_path, target_size=(48, 48), color_mode='grayscale')
img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict the emotion
predictions = model.predict(img_array)

# Get the predicted emotion label
emotion_label = np.argmax(predictions)  # Get the index of the highest probability
print(f"Predicted Emotion: {list(train_generator.class_indices.keys())[emotion_label]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted Emotion: happy
